## Setup Device & Environment

In [2]:
from datasets import load_dataset
import numpy as np

import torch
from transformers import AutoImageProcessor
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

import evaluate

In [3]:
device = torch.device('cuda')

device

device(type='cuda')

## Data Preparation

### Load Dataset

In [4]:
raw_dataset = load_dataset("imagefolder", data_dir="./datasets/chest_xray")

Resolving data files: 100%|██████████| 624/624 [00:00<?, ?it/s]


In [5]:
print(raw_dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5216
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 624
    })
})


### Setup Labels

In [6]:
labels = labels = raw_dataset["train"].features["label"].names
print(labels)

['NORMAL', 'PNEUMONIA']


In [7]:
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
  label2id[i] = label
  id2label[label] = i

In [8]:
print(label2id)
print(id2label)

{0: 'NORMAL', 1: 'PNEUMONIA'}
{'NORMAL': 0, 'PNEUMONIA': 1}


### Transforming Data

In [9]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

In [10]:
size = (image_processor.size["height"], image_processor.size["width"])
resizer = RandomResizedCrop(size)
normalize = Normalize(image_processor.image_mean, image_processor.image_std)

In [11]:
_transforms = Compose([resizer, ToTensor(), normalize])

In [12]:
def transforms(examples):
  examples["image"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
  # del examples["image"]
  return examples

In [13]:
dataset = raw_dataset.with_transform(transforms)

In [14]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5216
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 624
    })
})


### Preparing metrics for the model

In [15]:
accuracy = evaluate.load("accuracy")

In [16]:
def compute_metrics(eval_pred):
  predictions = np.argmax(eval_pred.predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

## Loading Best Model

In [18]:
from own_model import AutoCompositeModel
from training_own import get_model_params

PATH = './model_20231222_111631_1'

model = AutoCompositeModel()
model.load_state_dict(torch.load(PATH))
model.eval()
# saved
get_model_params(model.my_new_layers)

82770

## Training Setup

### Data Loaders

In [ ]:
training_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=16, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=16, shuffle=True)

### Loss Function

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

### Optimizer

In [ ]:
# Optimizers specified in the torch.optim package
optimizer = torch.optim.SGD(model.my_new_layers.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=2).to(device)

## Training Model

In [ ]:
from training_own import train_one_epoch, train_many_epochs

In [ ]:
model = model.to('cpu')
model.eval()

N_EXAMPLES = 25

cum_error = 0

testing_fragment = dataset['test'].shuffle(seed=1)[:N_EXAMPLES]
inputs = testing_fragment['image']
labels_true = testing_fragment['label']

for i in range(len(inputs)):
  input = inputs[i]
  label_true = labels_true[i]
  logits = model(input[None, ...]).detach().numpy()
  label_pred = np.argmax(logits)
  
  cum_error += abs(label_pred - label_true)
  print('index {}: true/predicted: {}/{}'.format(i, label_true, label_pred))
  
error = cum_error / N_EXAMPLES

accuracy = 1 - error

print('testing accuracy: {}'.format(accuracy))

In [ ]:
model = model.to(device)
train_many_epochs(
  2,
  model=model,
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  cuda_device=device,
  epoch_index=10,
  logging_frequency=25
)

In [ ]:
model = model.to('cpu')
model.eval()

N_EXAMPLES = 50

cum_error = 0

testing_fragment = dataset['test'].shuffle(seed=1)[:N_EXAMPLES]
inputs = testing_fragment['image']
labels_true = testing_fragment['label']

for i in range(len(inputs)):
  input = inputs[i]
  label_true = labels_true[i]
  logits = model(input[None, ...]).detach().numpy()
  label_pred = np.argmax(logits)
  
  cum_error += abs(label_pred - label_true)
  print('index {}: true/predicted: {}/{}'.format(i, label_true, label_pred))
  
error = cum_error / N_EXAMPLES

accuracy = 1 - error

print('testing accuracy: {}'.format(accuracy))

In [ ]:
model = model.to('cpu')
model.eval()

N_EXAMPLES = 50

cum_error = 0

testing_fragment = dataset['test'].shuffle(seed=1)[:N_EXAMPLES]
inputs = testing_fragment['image']
labels_true = testing_fragment['label']

for i in range(len(inputs)):
  input = inputs[i]
  label_true = labels_true[i]
  logits = model(input[None, ...]).detach().numpy()
  label_pred = np.argmax(logits)
  
  cum_error += abs(label_pred - label_true)
  print('index {}: true/predicted: {}/{}'.format(i, label_true, label_pred))
  
error = cum_error / N_EXAMPLES

accuracy = 1 - error

print('testing accuracy: {}'.format(accuracy))

###